In [1]:
! pip install pgmpy

  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.4 MB/s  0:00:00
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached networkx-3.5-py3-none-any.whl (2.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 43.7 MB/s  0:00:00eta 0:00:01
Using cached pytz-2025.2-p

In [2]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD



/opt/anaconda3/envs/comp_9814_w4/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
from pgmpy.models import DiscreteBayesianNetwork

dog_model = DiscreteBayesianNetwork([('FO','LO'),('FO','DO'),('BP','DO'),('DO','HB')])

In [20]:
cpd_fo = TabularCPD(variable='FO', variable_card=2, values=[[0.85], [0.15]],
state_names={'FO': ['False', 'True']})
cpd_bp = TabularCPD(variable='BP', variable_card=2, values=[[0.99], [0.01]],
state_names={'BP': ['False', 'True']})



In [21]:
cpd_lo = TabularCPD(variable='LO', variable_card=2,values=[[0.95, 0.4], [0.05, 0.6]],
    evidence=['FO'],evidence_card=[2],
    state_names={'LO': ['False', 'True'],'FO': ['False', 'True']})
cpd_do = TabularCPD(variable='DO', variable_card=2,
    values=[[0.70, 0.03, 0.10, 0.01], [0.30, 0.97, 0.90, 0.99]],
    evidence=['FO', 'BP'], evidence_card=[2, 2],
    state_names={'DO': ['False', 'True'],'FO': ['False', 'True'],'BP': ['False', 'True']})
cpd_hb = TabularCPD(variable='HB', variable_card=2,
    values=[[0.99, 0.30], [0.01, 0.70]],
    evidence=['DO'], evidence_card=[2],
    state_names={'HB': ['False', 'True'],'DO': ['False', 'True']})

In [22]:
dog_model.add_cpds(cpd_fo, cpd_bp, cpd_do, cpd_hb, cpd_lo)
dog_model.check_model()

True

In [23]:
dog_model.get_cpds()

[<TabularCPD representing P(FO:2) at 0x13babdcd0>,
 <TabularCPD representing P(BP:2) at 0x13babcda0>,
 <TabularCPD representing P(DO:2 | FO:2, BP:2) at 0x139478c20>,
 <TabularCPD representing P(HB:2 | DO:2) at 0x139479520>,
 <TabularCPD representing P(LO:2 | FO:2) at 0x13babfce0>]

In [24]:
print(dog_model.get_cpds('FO'))

+-----------+------+
| FO(False) | 0.85 |
+-----------+------+
| FO(True)  | 0.15 |
+-----------+------+


In [25]:
print(dog_model.get_cpds('BP'))

+-----------+------+
| BP(False) | 0.99 |
+-----------+------+
| BP(True)  | 0.01 |
+-----------+------+


In [26]:
print(dog_model.get_cpds('LO'))

+-----------+-----------+----------+
| FO        | FO(False) | FO(True) |
+-----------+-----------+----------+
| LO(False) | 0.95      | 0.4      |
+-----------+-----------+----------+
| LO(True)  | 0.05      | 0.6      |
+-----------+-----------+----------+


In [27]:
print(dog_model.get_cpds('DO'))

+-----------+-----------+-----------+-----------+----------+
| FO        | FO(False) | FO(False) | FO(True)  | FO(True) |
+-----------+-----------+-----------+-----------+----------+
| BP        | BP(False) | BP(True)  | BP(False) | BP(True) |
+-----------+-----------+-----------+-----------+----------+
| DO(False) | 0.7       | 0.03      | 0.1       | 0.01     |
+-----------+-----------+-----------+-----------+----------+
| DO(True)  | 0.3       | 0.97      | 0.9       | 0.99     |
+-----------+-----------+-----------+-----------+----------+


In [28]:
print(dog_model.get_cpds('HB'))

+-----------+-----------+----------+
| DO        | DO(False) | DO(True) |
+-----------+-----------+----------+
| HB(False) | 0.99      | 0.3      |
+-----------+-----------+----------+
| HB(True)  | 0.01      | 0.7      |
+-----------+-----------+----------+


In [29]:
from pgmpy.inference import VariableElimination
dog_infer = VariableElimination(dog_model)

In [30]:
joint_dist_di = dog_infer.query(variables=['FO', 'BP', 'LO', 'DO', 'HB'], joint=True)
print(joint_dist_di)

+-----------+-----------+-----------+-----------+-----------+-----------------------+
| FO        | BP        | LO        | DO        | HB        |   phi(FO,BP,LO,DO,HB) |
+===========+===========+===========+===========+===========+=======================+
| FO(False) | BP(False) | LO(False) | DO(False) | HB(False) |                0.5540 |
+-----------+-----------+-----------+-----------+-----------+-----------------------+
| FO(False) | BP(False) | LO(False) | DO(False) | HB(True)  |                0.0056 |
+-----------+-----------+-----------+-----------+-----------+-----------------------+
| FO(False) | BP(False) | LO(False) | DO(True)  | HB(False) |                0.0719 |
+-----------+-----------+-----------+-----------+-----------+-----------------------+
| FO(False) | BP(False) | LO(False) | DO(True)  | HB(True)  |                0.1679 |
+-----------+-----------+-----------+-----------+-----------+-----------------------+
| FO(False) | BP(False) | LO(True)  | DO(False) | HB(F

In [31]:
dog_q2_dist = dog_infer.query(variables=['DO'], evidence={'BP': 'True'})
print(dog_q2_dist)

+-----------+-----------+
| DO        |   phi(DO) |
+===========+===========+
| DO(False) |    0.0270 |
+-----------+-----------+
| DO(True)  |    0.9730 |
+-----------+-----------+


In [32]:
dog_q3_dist = dog_infer.query(variables=['BP'], evidence={'DO': 'False'})
print(dog_q3_dist)

+-----------+-----------+
| BP        |   phi(BP) |
+===========+===========+
| BP(False) |    0.9996 |
+-----------+-----------+
| BP(True)  |    0.0004 |
+-----------+-----------+


In [33]:
dog_hb_dist = dog_infer.query(variables=['HB'], evidence={'FO': 'False', 'BP': 'True', 'DO': 'True'})
print(dog_hb_dist)

+-----------+-----------+
| HB        |   phi(HB) |
+===========+===========+
| HB(False) |    0.3000 |
+-----------+-----------+
| HB(True)  |    0.7000 |
+-----------+-----------+
